Import necessary files and the path of our files

In [17]:
import math
import json
import os,glob
import geonamescache
folder_path = 'D:\\COVID-19-TweetIDs\\2020-01\\New folder\\22'
gc = geonamescache.GeonamesCache()

1. Reverse geocode

findlonglat is the function that find citycode from longitude and latitude and return a dictionary with key: citycode, value: relation information

In [18]:
city = gc.get_cities()
longlat = []
for k in city.keys():
    lst = []
    lst.append(city.get(k).get("longitude"))
    lst.append(city.get(k).get("latitude"))
    lst.append(city.get(k).get('geonameid'))
    longlat.append(lst)
tuple(longlat) #a tuple of all cities listed by geonamescache
def get_distance(data1, data2):
    points = zip(data1, data2)
    diffs_squared_distance = [pow(a - b, 2) for (a, b) in points]
    return math.sqrt(sum(diffs_squared_distance))
def findlonglat(a,b):
    found = []
    smallest = get_distance(b,a[0][0:2])
    for i in a:
        comparelist = i[0:2]
        if get_distance(b,comparelist) <= smallest:
            smallest = get_distance(b,comparelist)
            found = list(i)
    citycode = str(found[2])
    return {citycode:city[citycode]}

2. Look up city in geonamescache

findloc is the function that looks for the city in geonamescache data and return a dictionary with key: citycode and values: related information

In [19]:
def findloc(str):
    cities = str.split(",")[0].title() #use.title to make cities like hong kong to Hong Kong
    ct = gc.get_cities_by_name(cities)
    if (len(ct) == 1):
        return(ct[0]) #if return ct, it is a list of one member so lead to inconsistency of type
    elif ',' in str:
        loc2 = str.split(',')[1].strip()
        #print(loc2)
        for c in ct:
            if(loc2 in json.dumps(c)) :
                return(c)

In [23]:
a = []
location = {}   # dictionary of location: key is cityID, value is city information coming service
count = {}  # dictionary of count of tweet per location: key is cityID and value is number of tweet

for filename in glob.glob(os.path.join(folder_path, '*.jsonl')):
    fhand = open(filename, 'r',encoding="UTF-8")
    for line in fhand:
        tweet = json.loads(line)
        try:
            geo = tweet['coordinates']
            if geo != None: #Use coordinates to find location first (findlonglat)
                geo = tweet['coordinates']["coordinates"]
                loc = findlonglat(longlat,geo)
            else: #Then try to use user's location (findloc)
                loc = findloc(tweet['user']['location'])
            location.update(loc) #update loc info into the location dictionary
            for key,value in loc.items():
                if(key not in count):
                    count.update({key:1})   #update in the count dictionary, for places with 1 tweet
                elif(key in count):
                    count.update({key:(count[key]+1)})     #update in the count dictionary, if cityID is repeated,just increase the value by 1       
            
            #this step to compare the userlocation vs location that we found. 
            """
            user_name = tweet['user']["screen_name"]
            user_city_name = loc[key]["name"]
            user_country_name = loc[key]['countrycode']
            print(user_name, tweet['user']['location'], user_city_name, user_country_name)
            """
        
        except:
            continue
            
#output the two dictionaries if necessary
#print(location) 
#print(count)